<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/diffusers1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers

In [ ]:
!pip3 install torch torchvision torchaudio -U


In [ ]:
!pip install -q hidiffusion
!pip install -q diffusers==0.25.0 transformers==4.27.4 accelerate==0.18.0 xformers==0.0.16rc425

In [ ]:

from hidiffusion import apply_hidiffusion, remove_hidiffusion
from diffusers import StableDiffusionXLPipeline, DDIMScheduler
import torch
pretrain_model = "stabilityai/stable-diffusion-xl-base-1.0"
scheduler = DDIMScheduler.from_pretrained(pretrain_model, subfolder="scheduler")
pipe = StableDiffusionXLPipeline.from_pretrained(pretrain_model, scheduler = scheduler, torch_dtype=torch.float16, variant="fp16").to("cuda")
pipe.enable_model_cpu_offload()
pipe.enable_xformers_memory_efficient_attention()
pipe.enable_vae_tiling()
# Apply hidiffusion with a single line of code.
apply_hidiffusion(pipe)
prompt = "thick strokes, bright colors, an exotic fox, cute, chibi kawaii. detailed fur, hyperdetailed , big reflective eyes, fairytale, artstation,centered composition, perfect composition, centered, vibrant colors, muted colors, high detailed, 8k."
negative_prompt = "blurry, ugly, duplicate, poorly drawn, deformed, mosaic"
image = pipe(prompt, guidance_scale=7.5, height=2048, width=2048, eta=1.0, negative_prompt=negative_prompt).images[0]
image

In [ ]:
import torch
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

pipeline = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16"
)

pipeline.enable_model_cpu_offload()



image = load_image("https://radames-enhance-this-hidiffusion-sdxl.hf.space/file=/tmp/gradio/9c23bab0cd596a34f47a1867f205f82a634b5a2b287f4be8c6f587f495a71ce5/output_433.png")
image = image.resize((512, 512))

generator = torch.manual_seed(42)
frames = pipeline(image, decode_chunk_size=2, generator=generator).frames[0]
export_to_video(frames, "generated.mp4", fps=10)